# Install All Necessary Packages as Shown

In [8]:
!pip install lumibot
!pip install finbert_utils
!pip install alpaca_trade_api

ERROR: Could not find a version that satisfies the requirement finbert_utils (from versions: none)
ERROR: No matching distribution found for finbert_utils


In [9]:
!pip install alpaca-trade-api==3.1.1

In [10]:
!pip install install torch torchvision torchaudio transformers

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.6 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 355.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.4 MB 544.7 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/2.4 MB 1.1 MB/s eta 0:00:03
   ---- ----------------------------------- 0.3/2.4 MB 1.5 MB/s eta 0:00:02
   ------ --------------------------------- 0.4/2.4 MB 1.7 MB/s eta 0:00:02
   -------- ------------------------------- 0.5/2.4 MB 1.9 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.4 MB 1.9 MB/s eta 0:00:01
   ------------ --------------------------- 0.7/2.4 MB 2.0 MB/s eta 0:00:01
   -------------- -----

!pip install --upgrade transformers
!pip install pyyaml
!pip install huggingface_hub

# Trading Bot Algorithm

In [ ]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api.rest import REST
from pandas import Timedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

API_KEY = "PK60FWYZ61C9WLI23ROO"
API_SECRET = "cZcpcehSeOk7Ghq97gXEqKDZIBBigLdwPToKd961"
BASE_URL = "https://paper-api.alpaca.markets"

ALPACA_CREDS = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)
labels = ["positive", "negative", "neutral"]

def estimate_sentiment(news):
    if news:
        tokens = tokenizer(news, return_tensors="pt", padding=True).to(device)
        result = model(tokens["input_ids"], attention_mask=tokens["attention_mask"])["logits"]
        result = torch.nn.functional.softmax(torch.sum(result, 0), dim=-1)
        probability = result[torch.argmax(result)].item()
        sentiment = labels[torch.argmax(result)]
        return probability, sentiment
    else:
        return 0, labels[-1]

class MLTrader(Strategy):
    def initialize(self, symbol: str = "SPY", cash_at_risk: float = 0.5):
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.cash_at_risk = cash_at_risk
        self.api = REST(API_KEY, API_SECRET, BASE_URL)

    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)
        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_sentiment(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, start=three_days_prior, end=today)
        news = [ev._raw["headline"] for ev in news]
        probability, sentiment = estimate_sentiment(news)
        return probability, sentiment

    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        probability, sentiment = self.get_sentiment()
        if cash > last_price:
            if sentiment == "positive" and probability > .999: 
                if self.last_trade == "sell":
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    take_profit_price=last_price * 1.20,
                    stop_loss_price=last_price * 0.95
                )
                self.submit_order(order)
                self.last_trade = "buy"
            elif sentiment == "negative" and probability > .999: 
                if self.last_trade == "buy":
                    self.sell_all()
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "sell",
                    take_profit_price=last_price * 0.8,
                    stop_loss_price=last_price * 1.05
                )
                self.submit_order(order)
                self.last_trade = "sell"

start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 1, 1)
broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=broker, parameters={"symbol": "SPY", "cash_at_risk": 0.5})
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol": "SPY"}
)